In [1]:
import cv2
import librosa
import numpy as np
from moviepy.editor import VideoFileClip
from moviepy.audio.AudioClip import AudioArrayClip

In [2]:
def remove_noise(audio, noise_threshold):
    return librosa.effects.trim(audio, top_db=noise_threshold)[0]

def shift_pitch(audio, shift_factor):
    return librosa.effects.pitch_shift(
        audio, sr=audio.shape[0], n_steps=shift_factor,
        bins_per_octave=10
    )

def cut_high_frequencies(audio, cutoff_freq):
    audio_fft = np.fft.fft(audio)
    audio_fft[int(cutoff_freq):] = 0
    filtered_audio = np.fft.ifft(audio_fft)
    
    return np.real(filtered_audio)

def process_video(input_video_path, output_video_path,
                  noise_threshold=20, shift_factor=12, cutoff_freq=15000):
    # Load the video and extract audio
    clip = VideoFileClip(input_video_path)
    audio = clip.audio.to_soundarray()[:, 0]  # Select the first channel (mono)

    audio = audio / np.max(np.abs(audio))
    audio = cut_high_frequencies(audio, cutoff_freq)
    audio = shift_pitch(audio, shift_factor)

    audio = AudioArrayClip(np.expand_dims(audio, axis=1), fps=clip.audio.fps)
    
    clip = clip.set_audio(audio)
    clip.write_videofile(output_video_path, codec="libx264", audio_codec="aac")

input_path = 'noise2_20230603_163513.mp4'
output_path = 'test.mp4'
process_video(input_path, output_path)

Moviepy - Building video test.mp4.
MoviePy - Writing audio in testTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video test.mp4



t:  58%|█████████▊       | 1613231/2778300 [1:19:57<57:45, 336.18it/s, now=None]